# Data Wrangling - QC

## Data Exploration and Setup
Primarily follow QC protocol from Chuck's analysis of the PFC dataset [here](https://github.com/ListerLab/pfc_development/blob/master/snRNAseq/notebooks/02__clean-and-filter.ipynb) - add in any steps that might seem necessary from further research.\
Also guided by the [single cell best practices](https://www.sc-best-practices.org/introduction/prior_art.html) book by Theis Lab.

In [1]:
import anndata as ad
import pandas as pd
import numpy as np

herring_data = ad.read_h5ad('/group/ll005/cmcphan/herring_data/Processed_data_RNA-all_full-counts-and-downsampled-CPM.h5ad')
sepp_data_human = ad.read_h5ad('/group/ll005/cmcphan/sepp_data/sepp_human.h5ad')
sepp_data_mouse = ad.read_h5ad('/group/ll005/cmcphan/sepp_data/sepp_mouse.h5ad')
sepp_data_opossum = ad.read_h5ad('/group/ll005/cmcphan/sepp_data/sepp_opossum.h5ad')
zhu_data_bulk = ad.read_h5ad('/group/ll005/cmcphan/zhu_data/zhu_bulk.h5ad')

In [2]:
herring_data

AnnData object with n_obs × n_vars = 154748 × 26747
    obs: 'batch', 'RL#', 'age', 'chem', 'concat_id', 'numerical_age', 'stage_id', 'Sex', 'Race', 'PMI', 'Brain Regions*', 'Cause of Death', 'ICD-10 Code', 'ICD-10 category', 'Oxygen/No Oxygen', 'Date-of-Collection', 'Collection_year', 'Library Prep Date', 'Library Prep Lot', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'doublet_score', 'log10_gene_counts', 'log10_UMI_counts', 'percent_mito', 'percent_ribo', 'n_counts', 'leiden', 'mat/dev', 'cell_type', 'major_clust', 'sub_clust', 'combined-leiden', 'Astro_GFAP_dev-traj', 'Astro_SLC1A2_dev-traj', 'L2_CUX2_LAMP5_dev-traj', 'L3_CUX2_PRSS12_dev-traj', 'L4_RORB_LRRK1_dev-traj', 'L4_RORB_MET_dev-traj', 'L4_RORB_MME_dev-traj', 'L5-6_THEMIS_CNR1_dev-traj', 'L5-6_THEMIS_NTNG2_dev-traj', 'L5-6_TLE4_HTR2C_dev-traj', 'L5-6_TLE4_SCUBE1_dev-traj', 'L5-6_TLE4_SORCS1_dev-traj', 'Micro_dev-traj', 'OPC_dev-traj', 'OPC_MBP_dev-traj', 'Oligo_dev-traj', 'Vas_CLDN5_

In [3]:
herring_data.obs[['chem', 'age']].drop_duplicates()

,chem,age
AAACCTGAGAGTCGGT-RL1612_34d_v2,v2,34d
AAACCTGAGAAGGTTT-RL1613_2yr_v2,v2,2yr
AAACCTGAGTACGCGA-RL1614_8yr_v2,v2,8yr
AAACCCAAGTACAACA-RL1777_2d_v3,v3,2d
AAACCCAAGTAGATCA-RL1786_2yr_v3,v3,2yr
AAACCCAAGAGGCCAT-RL2100_86d_v3,v3,86d
AAACCCAAGGACGGAG-RL2102_16yr_v3,v3,16yr
AAACCCAAGAGTCTTC-RL2103_ga22_v3,v3,ga22
AAACCCAAGGCCACCT-RL2104_118d_v3,v3,118d
AAACCCAAGATGTTCC-RL2105_627d_v3,v3,627d


A few of the batches use the older v2 chemistry, which apparently show differences in MT/Ribo content and read counts per cell, but shouldn't affect the actual results too much. Just something to keep in mind for the QC, e.g. might see more cells filtered out of these batches. 

In [4]:
sepp_data_human

AnnData object with n_obs × n_vars = 180956 × 27715
    obs: 'orig_cluster', 'orig_sub_cluster', 'broad_lineage', 'cell_type', 'dev_state', 'subtype', 'precisest_label', 'species', 'Tissue', 'TissueID', 'batch', 'Capture.System', 'UMAP1', 'UMAP2', 'Stage', 'stage.ord', 'Stage_exact', 'size_factor'
    var: 'ensembl_gene_id', 'gene_symbol'

In [5]:
sepp_data_human.obs[['Capture.System', 'Stage_exact']].drop_duplicates()

,Capture.System,Stage_exact
SN003_HUM_CS22_CTAGCCTGTCAGATAA-1,Chromium,CS22
SN022_HUM_11wpc_GGCGACTGTCTCGTTC-1,Chromium,11 wpc
SN035_HUM_newborn_ACAGCTACAAGTCTAC-1,Chromium,newborn
SN060_HUM_infant_ATTTCTGCATGGATGG-1,Chromium,infant
SN080_HUM_toddler_TACACGACAGACACTT-1,Chromium,toddler
SN095_HUM_09wpc_ATTGGACGTTATTCTC-1,Chromium,9 wpc
SN097_HUM_17wpc_AAACCTGTCCAATGGT-1,Chromium,17 wpc
SN105_HUM_CS22_CTCCTTTCACCTGCGA-1,Chromium_v3,CS22
SN120_HUM_09wpc_TACTTACTCCTTCAGC-1,Chromium_v3,9 wpc
SN121_HUM_11wpc_TTCTTCCGTCTCTCAC-1,Chromium_v3,11 wpc


Similarly, Sepp datasets contain samples using v2 and v3 chemistries (as per extended figure 1). Most opossum samples were only analyzed with v3. 

In [6]:
sepp_data_human.obs[['batch', 'Stage', 'Stage_exact', 'stage.ord']].drop_duplicates().sort_values(by='stage.ord')

,batch,Stage,Stage_exact,stage.ord
SN134_HUM_CS18_CTCTCGAAGTGACCTT-1,SN134,7 wpc,CS18,02_7 wpc
SN170_HUM_CS18_CCTCAACAGTCTAACC-1,SN170,7 wpc,CS18,02_7 wpc
SN232_HUM_CS19HB_CCACTTGGTACGTACT-1,SN232,7 wpc,CS19,02_7 wpc
SN003_HUM_CS22_CTAGCCTGTCAGATAA-1,SN003,8 wpc,CS22,04_8 wpc
SN033_HUM_CS22_ATTTCTGCATGCCTTC-1,SN033,8 wpc,CS22,04_8 wpc
SN030_HUM_CS22_CACACAACATAAAGGT-1,SN030,8 wpc,CS22,04_8 wpc
SN034_HUM_CS22_GCAAACTTCATAACCG-1,SN034,8 wpc,CS22,04_8 wpc
SN026_HUM_CS22_CGAACATAGAAGGACA-1,SN026,8 wpc,CS22,04_8 wpc
SN021_HUM_CS22_CACAGTATCTTGTACT-1,SN021,8 wpc,CS22,04_8 wpc
SN009_HUM_CS22_ACTGAGTTCATTTGGG-1,SN009,8 wpc,CS22,04_8 wpc


In [7]:
zhu_data_bulk

AnnData object with n_obs × n_vars = 826 × 27932
    obs: 'Species', 'Brain', 'Sex', 'Region', 'NCXRegion', 'Age', 'Days', 'Period', 'Predicted age (PC Days)', 'Predicted period', 'Sequencing site', 'RNA extraction', 'Platform', 'RIN', 'Total reads', 'Uniquely mapped reads', 'Multiple reads', 'Uniquely mapped to ChrM'
    var: 'ensembl_id', 'non_unique_names'

In [8]:
zhu_data_bulk.obs[['Age', 'Days', 'Period', 'Platform']].drop_duplicates()

,Age,Days,Period,Platform
Sample,,,,
RMB209.AMY,E60,60.0,NaN,HiSeq
RMB193L.CBC,E81,81.0,NaN,GAIIx
RMB202.AMY,E82,82.0,NaN,HiSeq
RMB227.MD,E80,80.0,NaN,HiSeq
RMB296.AMY,E111,111.0,NaN,HiSeq
RMB233.AMY,E110,110.0,NaN,HiSeq
RMB200L.HIP,E110,110.0,NaN,GAIIx
RMB207.AMY,P2,166.0,NaN,HiSeq
RMB201L.AMY,P0,164.0,NaN,GAIIx


Sepp datasets are missing a whole bunch of metadata contained in the supplementary material, including the actual sample ages instead of just their stage labels (for those that have this information...) - add this back in.

In [9]:
sepp_metadata = pd.read_excel('/group/ll005/cmcphan/sepp_data/sepp_metadata_adj.xlsx')
sepp_metadata

,LibraryID,Sample source,Species,Individual ID,Sex,Stage,Age,Tissue,Fragment,RQN cytoplasm,...,SequencingID3,%inLane3,Intronic reads (%),Antisense reads (%),Number of nuclei,Median nGenes,Median nUMI,Median nGenes exons,Median nUMI exons,Notes
0,SN170,HDBR,HUM,14143,M,CS18 (7 wpc),7wpc,cerebellum,half,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SN134,HDBR,HUM,14068,M,CS18 (7 wpc),7wpc,cerebellum,fragment,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SN232,HDBR,HUM,13934,F,CS19 (7 wpc),7wpc,cerebellum,whole,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SN003,HDBR,HUM,11858,M,CS22 (8 wpc),8wpc,cerebellum,representative part,9.7,...,NaN,NaN,33.4,23.5,6000.0,1394.0,1957.0,504,715,SN003_280218 and SN009_220318 are from the sam...
4,SN009,HDBR,HUM,11858,M,CS22 (8 wpc),8wpc,cerebellum,representative part,9.7,...,NaN,NaN,35.7,24.9,3700.0,1940.0,2845.0,629,882,SN003_280218 and SN009_220318 are from the sam...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,SN082,Texas,OPO,070714 M1,M,P42,P42,cerebellum,half,5.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,SN210,Texas,OPO,020714 M3,M,P60,P60,cerebellum,half,8.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,SN204,Texas,OPO,020714 F3,F,P60,P60,cerebellum,whole,8.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,SN148,Berlin,OPO,220119a M4,M,adult (14 months),14m,cerebellum,half,6.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Numerical age for fetal samples are negative based on 40 week gestation, with -280/365 being conception and 0 being birth
def calc_fetal( ga):
    return( ( ( ga * 7) - ( 40 * 7)) / 365)

import re, math
# convert all ages to a common numerical scale and add feature, i.e. years
def numerical_age( adata):
    # get numerical age
    str_age = sepp_metadata['Age'].drop_duplicates().to_list()
    num_age = []
    for age_itr in str_age:
        if pd.isnull(age_itr): # We have a NaN which won't be picked up by the regex
            num_age.append(math.nan)
            continue
        digit = float(re.findall(r'\d+\.*\d*', age_itr)[0]) # Look for any integer or decimal value
        if( "ga" in age_itr):
            num_age.append( calc_fetal( digit))
        elif( "d" in age_itr):
            num_age.append( digit / 365)
        elif( "yr" in age_itr):
            num_age.append( digit)
        elif( "wpc" in age_itr):
            num_age.append( calc_fetal( digit + 2)) # Adjusted to account for difference between age post conception and gestational age
        elif( "w" in age_itr):
            num_age.append( digit * 7 / 365) # Approximation based on information given
        elif( "m" in age_itr):
            num_age.append( 1 / (12 / digit)) # Approximation based on information given
        elif( "EM" in age_itr): # Embryonic day (Mouse)
            num_age.append( (digit - 20) / 365) # Based on 20 day Mus musculus gestational period
        elif( "P" in age_itr):
            num_age.append( digit / 365)
        elif( "EO" in age_itr): # Embryonic day (Opossum)
            num_age.append( -1 / 365) # Arbitrary negative value to indicate pre-birth
            # Opossum gestational period is 12-14days, our only embryonic opossum samples are E14
            # A calculation as above would put these at 0, which implies birth, so keep them negative but small
        else:
            print( age_itr[1] + " of unknown age type, numerical value set to 0.0")
    # add feature to anndata
    adata.obs['numerical_age'] = [num_age[str_age.index(ii)] for ii in adata.obs['age']]
    return

In [ ]:
# Add in QC features from metadata that were omitted in provided datasets
# Keep any features that could be confounding factors in downstream analyses
for dataset in [sepp_data_human, sepp_data_mouse, sepp_data_opossum]:
    sex = []
    age = []
    fragment = []
    rqn_cytoplasm = []
    nuclei_preparation = []
    chromium_version = []
    for sample in dataset.obs['batch']:
        sex.append(sepp_metadata.loc[sepp_metadata['LibraryID'] == sample]['Sex'].array[0])
        age.append(sepp_metadata.loc[sepp_metadata['LibraryID'] == sample]['Age'].array[0])
        fragment.append(sepp_metadata.loc[sepp_metadata['LibraryID'] == sample]['Fragment'].array[0])
        rqn_cytoplasm.append(str(sepp_metadata.loc[sepp_metadata['LibraryID'] == sample]['RQN cytoplasm'].array[0]))
        # Float needs to be casted to str otherwise the write function won't work later
        nuclei_preparation.append(sepp_metadata.loc[sepp_metadata['LibraryID'] == sample]['Nuclei preparation'].array[0])
        chromium_version.append(sepp_metadata.loc[sepp_metadata['LibraryID'] == sample]['Chromium version'].array[0])
    dataset.obs['sex'] = sex
    dataset.obs['age'] = age
    dataset.obs['fragment'] = fragment
    dataset.obs['rqn_cytoplasm'] = rqn_cytoplasm
    dataset.obs['nuclei_preparation'] = nuclei_preparation
    dataset.obs['chromium_version'] = chromium_version
    # Calculate and add numerical age as well
    numerical_age(dataset)

In [ ]:
sepp_data_human.write_h5ad('/group/ll005/cmcphan/sepp_data/sepp_human.h5ad')
sepp_data_mouse.write_h5ad('/group/ll005/cmcphan/sepp_data/sepp_mouse.h5ad')
sepp_data_opossum.write_h5ad('/group/ll005/cmcphan/sepp_data/sepp_opossum.h5ad')